In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input, Multiply
from tensorflow.keras.activations import relu
from DataLoader import DataLoader
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model

import numpy as np

In [2]:
training_generator = DataLoader(256, "train")
val_generator = DataLoader(256, "val")

In [6]:
lossHistory = []
valHistory = []
class NnomCallbacks(tf.keras.callbacks.Callback):
  def on_batch_end(self, batch, logs=None):
    lossHistory.append(logs['accuracy'])
    if batch and batch % 1000 == 0:
      # pred = self.model.predict(val_generator)
      # loss = (pred.flatten() - val_generator.labels.flatten()[0:len(val_generator)*64])**2
      # loss = np.sum(loss) / len(loss)
      # valHistory.append(loss)

      # plt.plot(valHistory)
      # plt.savefig("val.png", dpi=200)
      # plt.clf()

      plt.plot(lossHistory[20:])
      plt.savefig("loss.png", dpi=200)
      plt.clf()
      
      # print(" val_loss : {}".format(loss))

      # self.model.save("models/model-{}-{}".format(len(valHistory), int(loss*10000)))

In [7]:
i_main = Input((12*64,), sparse=True)
i_legal = Input((384,), sparse=True)

layerStack = Dense(512, activation='relu')(i_main)
layerStack = Dense(384, activation='sigmoid')(layerStack)
layerStack = Multiply()([layerStack, i_legal])

model = Model(inputs=[i_main, i_legal], outputs=layerStack)
model.compile(optimizer=tf.keras.optimizers.Adam(), loss='mse', metrics=['accuracy'])

In [8]:
model = tf.keras.models.load_model("production/")

In [8]:
model.fit(training_generator, validation_data=val_generator, epochs=1, callbacks=[NnomCallbacks()])

 7340/46117 [===>..........................] - ETA: 6:59 - loss: 0.0022 - accuracy: 0.2802

In [ ]:
model.summary()

In [14]:
model.save("production/")

INFO:tensorflow:Assets written to: production/assets


INFO:tensorflow:Assets written to: production/assets
/home/levigibson/.local/lib/python3.8/site-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)
/home/levigibson/.local/lib/python3.8/site-packages/keras/saving/saved_model/layer_serialization.py:112: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  return generic_utils.serialize_keras_object(obj)


In [ ]:
import time
model.save("models/model-{}".format(int(time.time())))